In [ ]:
# Goal: Let's forecast when College will no longer be a worthwhile investment!

In [42]:
# My API Key
# api_key=2WELGSTK4bPqx8bOadxd5ljVOvvoqb62QvDGS5rf
# Example:
# https://developer.nrel.gov/api/alt-fuel-stations/v1/nearest.json?api_key=2WELGSTK4bPqx8bOadxd5ljVOvvoqb62QvDGS5rf&location=Denver+CO

In [43]:
# https://api.data.gov/ed/collegescorecard/v1/schools?

In [44]:
# 7593 schools
# = 380 pages @ 20 results per page

In [1]:
import requests
import pandas as pd
import numpy as np
import time

from collections import Counter

from easymoney.money import EasyPeasy

In [2]:
base_url = 'https://api.data.gov/ed/collegescorecard/v1/schools?'
key = 'api_key=2WELGSTK4bPqx8bOadxd5ljVOvvoqb62QvDGS5rf'

In [3]:
years = np.arange(1996, 2016, 1)
pages = np.arange(0,381,1)

In [38]:
root_fields = ['id']

# school.field
school_fields = ['name',
                 'state',
                 'degrees_awarded.predominant', # 0 to 4, with 4 being graduate degree
                 'under_investigation', # 1 for yes, 0 for no
                 'main_campus', # 1 for yes, 0 for no
                 'branches', # number of branch campuses
                 'degrees_awarded.highest', # 0 to 4
                 'ownership', # 1, 2 or 3 for public, private non-proft, private for-profit
                 'locale', # categorical of city
                 'carnegie_basic', # categorical of type
                 'carnegie_undergrad', # categorical of enrollment
                 'carnegie_size_setting', # categorical of size and setting
                 'online_only', # 1 for online only, 0 for otherwise
                 'operating', # 1 for operating, 0 for closed
                 'tuition_revenue_per_fte', # Net revenue per student
                 'instructional_expenditure_per_fte', # Net expenditure per student
                 'faculty_salary', # Average faculty salary
                 'ft_faculty_rate', # Proportion of faculty that is full-time
                 'institutional_characteristics.level' # 1, 2 or 3 for 4-year, 2-year, less than 2-year
                ]

# "year".admissions.field
admissions_fields = ['admission_rate.overall' # overall admission rate
                   ]

# Had to trim these, "414 Request-URI Too Large"
# "year".student.field
student_fields = ['size', # Total size (excluding 2001)
                  #'enrollment.all', # Total size (2001)
                  'share_lowincome.0_30000', # Average
                  'share_middleincome.30001_48000',
                  'share_middleincome.48001_75000',
                  'share_highincome.75001_110000',
                  'share_highincome.110001plus',
                  #'demographics.avg_family_income', # In 2015 dollars
                  #'demographics.median_family_income', # In 2015 dollars
                  #'demographics.median_hh_income',
                  'undergrads_with_pell_grant_or_federal_student_loan', # total number of
                 ]

# Had to trim these, "414 Request-URI Too Large"
# "year".cost.field
cost_fields = [#'avg_net_price.public', 
               #'avg_net_price.private',
               'net_price.public.by_income_level.0-30000', # Average cost AFTER scholarships/aid
               #'net_price.public.by_income_level.30001-48000',
               'net_price.public.by_income_level.48001-75000',
               #'net_price.public.by_income_level.75001-110000',
               'net_price.public.by_income_level.110001-plus',
               'net_price.private.by_income_level.0-30000',
               #'net_price.private.by_income_level.30001-48000',
               'net_price.private.by_income_level.48001-75000',
               #'net_price.private.by_income_level.75001-110000',
               'net_price.private.by_income_level.110001-plus',
               #'title_iv.public.all', # Total number of students on title iv aid
               #'title_iv.private.all',
               #'title_iv.public.by_income_level.0-30000',
               #'title_iv.public.by_income_level.30001-48000',
               #'title_iv.public.by_income_level.48001-75000',
               #'title_iv.public.by_income_level.75001-110000',
               #'title_iv.public.by_income_level.110001-plus',
               #'title_iv.private.by_income_level.0-30000',
               #'title_iv.private.by_income_level.30001-48000',
               #'title_iv.private.by_income_level.48001-75000',
               #'title_iv.private.by_income_level.75001-110000',
               #'title_iv.private.by_income_level.110001-plus',
               #'attendance.academic_year', # Average cost
               #'attendance.program_year', # Average cost
               #'tuition.in_state',
               #'tuition.out_of_state',
               #'tuition.program_year'
                ]

# Had to trim these, "414 Request-URI Too Large"
# "year".aid.field                
aid_fields = [#'pell_grant_rate', # percent of all receiving federal aid
              #'federal_loan_rate', # percent of all undergrads receiving federal aid
              'loan_principal', # median amount of loan principal
              'median_debt.completers.overall',
              'median_debt.noncompleters',
              #'median_debt.income.0_30000',
              #'median_debt.income.30001_75000',
              #'median_debt.income.greater_than_75000',
              #'median_debt.number.overall', # Total number of students in median debt calc
              #'median_debt.number.income.0_30000', 
              #'median_debt.number.income.30001_75000',
              #'median_debt.number.income.greater_than_75000',
              #'median_debt.completers.monthly_payments',
              #'cumulative_debt.number', # Total number of students in cumulative debt calc
              'cumulative_debt.90th_percentile', # Cumulative debt
              'cumulative_debt.75th_percentile',
              'cumulative_debt.25th_percentile',
              'cumulative_debt.10th_percentile',
              #'students_with_any_loan', # percentage of students with any debt
              #'median_debt_suppressed.overall'
            ]

# "year".repayment.field
repayment_fields = [#'2_yr_default_rate',
                    #'3_yr_default_rate',
                    
                    #'repayment_cohort.1_year_declining_balance' # All rates
                    #'1_yr_repayment.completers_rate',
                    #'1_yr_repayment.noncompleters_rate',
                    #'1_yr_repayment.income.0_30000',
                    #'1_yr_repayment.income.30000_75000',
                    #'1_yr_repayment.income.greater_than_75000',
                    
                    'repayment_cohort.3_year_declining_balance',
                    '3_yr_repayment.completers_rate',
                    '3_yr_repayment.noncompleters_rate',
                    #'3_yr_repayment.income.0_30000',
                    #'3_yr_repayment.income.30000_75000',
                    #'3_yr_repayment.income.greater_than_75000',
                    
                    #'repayment_cohort.5_year_declining_balance'
                    #'5_yr_repayment.completers_rate',
                    #'5_yr_repayment.noncompleters_rate',
                    #'5_yr_repayment.income.0_30000',
                    #'5_yr_repayment.income.30000_75000',
                    #'5_yr_repayment.income.greater_than_75000',
                    
                    'repayment_cohort.7_year_declining_balance',
                    '7_yr_repayment.completers_rate',
                    '7_yr_repayment.noncompleters_rate',
                    #'7_yr_repayment.income.0_30000',
                    #'7_yr_repayment.income.30000_75000',
                    #'7_yr_repayment.income.greater_than_75000',
                   ]

# "year".earnings.field
earnings_fields = ['10_yrs_after_entry.not_working_not_enrolled.overall', # Total number of students
                   '10_yrs_after_entry.working_not_enrolled.mean_earnings',
                   '10_yrs_after_entry.median', # Median of above
                   '10_yrs_after_entry.mean_earnings.lowest_tercile',
                   '10_yrs_after_entry.mean_earnings.middle_tercile',
                   '10_yrs_after_entry.mean_earnings.highest_tercile']

In [39]:
# ONLY RUN THIS ONCE AFTER PREVIOUS!!!

fields_list1 = []
for i in school_fields:
    fields_list1.append('school.'+i)
school_fields = ','.join(fields_list1)

fields_list2 = []
for year in years:
    fields_list2.append(str(year) + '.admissions.'+ admissions_fields[0])
admissions_fields = ','.join(fields_list2)


fields_list3 = []
for year in years:
    for field in student_fields:
        fields_list3.append(str(year) + '.student.'+ field)
student_fields = ','.join(fields_list3)

fields_list4 = []
for year in years:
    for field in cost_fields:
        fields_list4.append(str(year) + '.cost.'+ field)
cost_fields = ','.join(fields_list4)

fields_list5 = []
for year in years:
    for field in aid_fields:
        fields_list5.append(str(year) + '.aid.'+ field)
aid_fields = ','.join(fields_list5)

fields_list6 = []
for year in years:
    for field in repayment_fields:
        fields_list6.append(str(year) + '.repayment.'+ field)
repayment_fields = ','.join(fields_list6)

fields_list7 = []
for year in years:
    for field in earnings_fields:
        fields_list7.append(str(year) + '.earnings.'+ field)
earnings_fields = ','.join(fields_list7)

## Basic School Info (Try later, reached limit)

In [40]:
# Basic school dataframe

school_df = pd.DataFrame(columns=fields_list1)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=' + school_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    school_df = pd.concat([school_df,this_df])
    time.sleep(1)

parsing page... 0


KeyError: 'results'

In [38]:
school_df.isnull().sum()

school.branches                                  0
school.carnegie_basic                          446
school.carnegie_size_setting                   446
school.carnegie_undergrad                      446
school.degrees_awarded.highest                   0
school.degrees_awarded.predominant               0
school.faculty_salary                         3139
school.ft_faculty_rate                        3656
school.institutional_characteristics.level       0
school.instructional_expenditure_per_fte       460
school.locale                                  446
school.main_campus                               0
school.name                                      0
school.operating                                 0
school.ownership                                 0
school.state                                     0
school.tuition_revenue_per_fte                 460
school.under_investigation                       0
dtype: int64

In [39]:
school_df.to_csv('./school_df.csv')

In [40]:
school_df.head()

,school.branches,school.carnegie_basic,school.carnegie_size_setting,school.carnegie_undergrad,school.degrees_awarded.highest,school.degrees_awarded.predominant,school.faculty_salary,school.ft_faculty_rate,school.institutional_characteristics.level,school.instructional_expenditure_per_fte,school.locale,school.main_campus,school.name,school.operating,school.ownership,school.state,school.tuition_revenue_per_fte,school.under_investigation
0,3,15,15,9,4,3,9382,0.7669,1,9275,11,1,University of South Florida-Main Campus,1,1,FL,6669,0
1,1,-3,-2,-2,1,1,NaN,NaN,3,3954,41,1,Radford M Locklin Technical Center,1,1,FL,965,0
2,1,23,15,5,3,2,7061,0.239,1,4541,13,1,St Petersburg College,1,1,FL,1994,0
3,1,7,15,5,3,2,6668,0.322,1,3972,21,1,Broward College,1,1,FL,1327,0
4,1,24,6,5,4,3,5449,0.4722,1,2554,21,1,Luther Rice College & Seminary,1,2,GA,6595,0


In [46]:
Counter(school_df['school.under_investigation'])

Counter({0: 7523, 1: 70})

## Admission Rate Info

In [64]:
admissions_df = pd.DataFrame(columns=fields_list2)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=school.name,' + admissions_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    admissions_df = pd.concat([admissions_df,this_df])
    time.sleep(1)

parsing page... 0
parsing page... 10
parsing page... 20
parsing page... 30
parsing page... 40
parsing page... 50
parsing page... 60
parsing page... 70
parsing page... 80
parsing page... 90
parsing page... 100
parsing page... 110
parsing page... 120
parsing page... 130
parsing page... 140
parsing page... 150
parsing page... 160
parsing page... 170
parsing page... 180
parsing page... 190
parsing page... 200
parsing page... 210
parsing page... 220
parsing page... 230
parsing page... 240
parsing page... 250
parsing page... 260
parsing page... 270
parsing page... 280
parsing page... 290
parsing page... 300
parsing page... 310
parsing page... 320
parsing page... 330
parsing page... 340
parsing page... 350
parsing page... 360
parsing page... 370
parsing page... 380


In [65]:
admissions_df.head()

,1996.admissions.admission_rate.overall,1997.admissions.admission_rate.overall,1998.admissions.admission_rate.overall,1999.admissions.admission_rate.overall,2000.admissions.admission_rate.overall,2001.admissions.admission_rate.overall,2002.admissions.admission_rate.overall,2003.admissions.admission_rate.overall,2004.admissions.admission_rate.overall,2005.admissions.admission_rate.overall,...,2007.admissions.admission_rate.overall,2008.admissions.admission_rate.overall,2009.admissions.admission_rate.overall,2010.admissions.admission_rate.overall,2011.admissions.admission_rate.overall,2012.admissions.admission_rate.overall,2013.admissions.admission_rate.overall,2014.admissions.admission_rate.overall,2015.admissions.admission_rate.overall,school.name
0,None,None,None,None,None,0.6687,0.6216,0.6176,0.5109,0.2831,...,0.4967,0.456,0.4244,0.4454,0.3805,0.4321,0.4524,0.5316,0.4464,University of South Florida-Main Campus
1,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Radford M Locklin Technical Center
2,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,St Petersburg College
3,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Broward College
4,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4878,1,Luther Rice College & Seminary


In [66]:
admissions_df.isnull().sum()

1996.admissions.admission_rate.overall    7593
1997.admissions.admission_rate.overall    7593
1998.admissions.admission_rate.overall    7593
1999.admissions.admission_rate.overall    7593
2000.admissions.admission_rate.overall    7593
2001.admissions.admission_rate.overall    5473
2002.admissions.admission_rate.overall    5200
2003.admissions.admission_rate.overall    5166
2004.admissions.admission_rate.overall    5197
2005.admissions.admission_rate.overall    5156
2006.admissions.admission_rate.overall    5094
2007.admissions.admission_rate.overall    5074
2008.admissions.admission_rate.overall    5076
2009.admissions.admission_rate.overall    5054
2010.admissions.admission_rate.overall    5420
2011.admissions.admission_rate.overall    5379
2012.admissions.admission_rate.overall    5401
2013.admissions.admission_rate.overall    5414
2014.admissions.admission_rate.overall    5419
2015.admissions.admission_rate.overall    5422
school.name                                  0
dtype: int64

In [67]:
# Let's drop those 5 years that have no values
admissions_df.drop(['1996.admissions.admission_rate.overall',
                    '1997.admissions.admission_rate.overall',
                    '1998.admissions.admission_rate.overall',
                    '1999.admissions.admission_rate.overall',
                    '2000.admissions.admission_rate.overall'], axis=1, inplace=True)

In [77]:
admissions_df['2015.admissions.admission_rate.overall'].median()

0.7134

In [78]:
admissions_df.to_csv('./admissions_df.csv')

In [96]:
len(fields_list3)

116380

## Student Info

In [99]:
student_df = pd.DataFrame(columns=fields_list3)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=school.name,' + student_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    student_df = pd.concat([student_df,this_df])
    time.sleep(1)

parsing page... 0
parsing page... 10
parsing page... 20
parsing page... 30
parsing page... 40
parsing page... 50
parsing page... 60
parsing page... 70
parsing page... 80
parsing page... 90
parsing page... 100
parsing page... 110
parsing page... 120
parsing page... 130
parsing page... 140
parsing page... 150
parsing page... 160
parsing page... 170
parsing page... 180
parsing page... 190
parsing page... 200
parsing page... 210
parsing page... 220
parsing page... 230
parsing page... 240
parsing page... 250
parsing page... 260
parsing page... 270
parsing page... 280
parsing page... 290
parsing page... 300
parsing page... 310
parsing page... 320
parsing page... 330
parsing page... 340
parsing page... 350
parsing page... 360
parsing page... 370
parsing page... 380


In [100]:
student_df.head()

,1996.student.share_highincome.110001plus,1996.student.share_highincome.75001_110000,1996.student.share_lowincome.0_30000,1996.student.share_middleincome.30001_48000,1996.student.share_middleincome.48001_75000,1996.student.size,1996.student.undergrads_with_pell_grant_or_federal_student_loan,1997.student.share_highincome.110001plus,1997.student.share_highincome.75001_110000,1997.student.share_lowincome.0_30000,...,2014.student.size,2014.student.undergrads_with_pell_grant_or_federal_student_loan,2015.student.share_highincome.110001plus,2015.student.share_highincome.75001_110000,2015.student.share_lowincome.0_30000,2015.student.share_middleincome.30001_48000,2015.student.share_middleincome.48001_75000,2015.student.size,2015.student.undergrads_with_pell_grant_or_federal_student_loan,school.name
0,None,None,None,None,None,24313,None,0.0770121,0.111983,0.480715,...,30279,30264,0.110213,0.101725,0.464213,0.178052,0.145797,30282,30155,University of South Florida-Main Campus
1,None,None,None,None,None,452,None,NaN,NaN,0.818182,...,318,824,NaN,NaN,0.807339,0.110092,NaN,315,982,Radford M Locklin Technical Center
2,None,None,None,None,None,16146,None,0.0141769,0.028777,0.699534,...,28003,31820,0.026875,0.049783,0.643435,0.176299,0.103608,27354,32681,St Petersburg College
3,None,None,None,None,None,22842,None,0.0138169,0.0328152,0.691631,...,39175,43883,0.012130,0.024474,0.688483,0.186790,0.088123,38474,44119,Broward College
4,None,None,None,None,None,449,None,NaN,NaN,NaN,...,490,511,NaN,NaN,0.551887,0.188679,0.193396,409,491,Luther Rice College & Seminary


In [101]:
student_df.isnull().sum()

1996.student.share_highincome.110001plus                           7593
1996.student.share_highincome.75001_110000                         7593
1996.student.share_lowincome.0_30000                               7593
1996.student.share_middleincome.30001_48000                        7593
1996.student.share_middleincome.48001_75000                        7593
1996.student.size                                                  3203
1996.student.undergrads_with_pell_grant_or_federal_student_loan    7593
1997.student.share_highincome.110001plus                           5056
1997.student.share_highincome.75001_110000                         4997
1997.student.share_lowincome.0_30000                               3390
1997.student.share_middleincome.30001_48000                        3951
1997.student.share_middleincome.48001_75000                        4353
1997.student.size                                                  3026
1997.student.undergrads_with_pell_grant_or_federal_student_loan 

In [102]:
student_df.to_csv('./student_df.csv')

## Cost Info

In [6]:
cost_df = pd.DataFrame(columns=fields_list4)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=school.name,' + cost_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    cost_df = pd.concat([cost_df,this_df])
    time.sleep(1)

parsing page... 0
parsing page... 10
parsing page... 20
parsing page... 30
parsing page... 40
parsing page... 50
parsing page... 60
parsing page... 70
parsing page... 80
parsing page... 90
parsing page... 100
parsing page... 110
parsing page... 120
parsing page... 130
parsing page... 140
parsing page... 150
parsing page... 160
parsing page... 170
parsing page... 180
parsing page... 190
parsing page... 200
parsing page... 210
parsing page... 220
parsing page... 230
parsing page... 240
parsing page... 250
parsing page... 260
parsing page... 270
parsing page... 280
parsing page... 290
parsing page... 300
parsing page... 310
parsing page... 320
parsing page... 330
parsing page... 340
parsing page... 350
parsing page... 360
parsing page... 370
parsing page... 380


In [13]:
list(cost_df.isnull().sum())

[7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 7593,
 5739,
 6268,
 5919,
 6007,
 6533,
 6177,
 5669,
 6244,
 5863,
 6002,
 6492,
 6157,
 3580,
 5521,
 4497,
 5711,
 6417,
 5978,
 3481,
 5532,
 4452,
 5715,
 6361,
 5963,
 3334,
 5498,
 4358,
 5703,
 6319,
 5901,
 3200,
 5433,
 4310,
 5699,
 6307,
 5844,
 3138,
 5455,
 4306,
 5693,
 6275,
 5843,
 0]

In [20]:
# 1996-2008 are all Null
cost_drop_cols = cost_df.columns[0:78]
cost_df.drop(cost_drop_cols, axis=1, inplace=True)

In [22]:
cost_df.to_csv('./cost_df.csv')

## Aid Info

In [23]:
aid_df = pd.DataFrame(columns=fields_list5)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=school.name,' + aid_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    aid_df = pd.concat([aid_df,this_df])
    time.sleep(1)

parsing page... 0
parsing page... 10
parsing page... 20
parsing page... 30
parsing page... 40
parsing page... 50
parsing page... 60
parsing page... 70
parsing page... 80
parsing page... 90
parsing page... 100
parsing page... 110
parsing page... 120
parsing page... 130
parsing page... 140
parsing page... 150
parsing page... 160
parsing page... 170
parsing page... 180
parsing page... 190
parsing page... 200
parsing page... 210
parsing page... 220
parsing page... 230
parsing page... 240
parsing page... 250
parsing page... 260
parsing page... 270
parsing page... 280
parsing page... 290
parsing page... 300
parsing page... 310
parsing page... 320
parsing page... 330
parsing page... 340
parsing page... 350
parsing page... 360
parsing page... 370
parsing page... 380


In [25]:
aid_df.isnull().sum()

1996.aid.cumulative_debt.10th_percentile    7593
1996.aid.cumulative_debt.25th_percentile    7593
1996.aid.cumulative_debt.75th_percentile    7593
1996.aid.cumulative_debt.90th_percentile    7593
1996.aid.loan_principal                     7593
1996.aid.median_debt.completers.overall     7593
1996.aid.median_debt.noncompleters          7593
1997.aid.cumulative_debt.10th_percentile    4456
1997.aid.cumulative_debt.25th_percentile    3966
1997.aid.cumulative_debt.75th_percentile    3966
1997.aid.cumulative_debt.90th_percentile    4456
1997.aid.loan_principal                     3764
1997.aid.median_debt.completers.overall     4672
1997.aid.median_debt.noncompleters          4726
1998.aid.cumulative_debt.10th_percentile    4349
1998.aid.cumulative_debt.25th_percentile    3894
1998.aid.cumulative_debt.75th_percentile    3894
1998.aid.cumulative_debt.90th_percentile    4349
1998.aid.loan_principal                     3745
1998.aid.median_debt.completers.overall     4561
1998.aid.median_debt

In [27]:
# Let's remove 1996 data since it's all null
aid_drop_cols = aid_df.columns[0:7]
aid_df.drop(aid_drop_cols, axis=1, inplace=True)

In [28]:
aid_df.to_csv('./aid_df.csv')

## Repayment Info

In [29]:
repayment_df = pd.DataFrame(columns=fields_list6)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=school.name,' + repayment_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    repayment_df = pd.concat([repayment_df,this_df])
    time.sleep(1)

parsing page... 0
parsing page... 10
parsing page... 20
parsing page... 30
parsing page... 40
parsing page... 50
parsing page... 60
parsing page... 70
parsing page... 80
parsing page... 90
parsing page... 100
parsing page... 110
parsing page... 120
parsing page... 130
parsing page... 140
parsing page... 150
parsing page... 160
parsing page... 170
parsing page... 180
parsing page... 190
parsing page... 200
parsing page... 210
parsing page... 220
parsing page... 230
parsing page... 240
parsing page... 250
parsing page... 260
parsing page... 270
parsing page... 280
parsing page... 290
parsing page... 300
parsing page... 310
parsing page... 320
parsing page... 330
parsing page... 340
parsing page... 350
parsing page... 360
parsing page... 370
parsing page... 380


In [33]:
repayment_df.isnull().sum()

1996.repayment.3_yr_repayment.completers_rate               7593
1996.repayment.3_yr_repayment.noncompleters_rate            7593
1996.repayment.7_yr_repayment.completers_rate               7593
1996.repayment.7_yr_repayment.noncompleters_rate            7593
1996.repayment.repayment_cohort.3_year_declining_balance    7593
1996.repayment.repayment_cohort.7_year_declining_balance    7593
1997.repayment.3_yr_repayment.completers_rate               7593
1997.repayment.3_yr_repayment.noncompleters_rate            7593
1997.repayment.7_yr_repayment.completers_rate               7593
1997.repayment.7_yr_repayment.noncompleters_rate            7593
1997.repayment.repayment_cohort.3_year_declining_balance    7593
1997.repayment.repayment_cohort.7_year_declining_balance    7593
1998.repayment.3_yr_repayment.completers_rate               7593
1998.repayment.3_yr_repayment.noncompleters_rate            7593
1998.repayment.7_yr_repayment.completers_rate               7593
1998.repayment.7_yr_repay

In [34]:
# 1996-2008 are all Null
repayment_drop_cols = repayment_df.columns[0:78]
repayment_df.drop(repayment_drop_cols, axis=1, inplace=True)

## Earnings Info (Try Later, reached limit)

In [37]:
earnings_df = pd.DataFrame(columns=fields_list7)
for page in pages:
    if page % 10 == 0:
        print('parsing page...', page)
    URL = base_url + '_fields=school.name,' + earnings_fields + '&' + key + '&' + '_page=' + str(page)
    response = requests.get(URL)
    json_results = response.json()
    this_df = pd.DataFrame(json_results['results'])
    earnings_df = pd.concat([earnings_df,this_df])
    time.sleep(1)

parsing page... 0
parsing page... 10
parsing page... 20
parsing page... 30
parsing page... 40
parsing page... 50
parsing page... 60
parsing page... 70
parsing page... 80
parsing page... 90
parsing page... 100
parsing page... 110
parsing page... 120
parsing page... 130
parsing page... 140
parsing page... 150
parsing page... 160
parsing page... 170
parsing page... 180
parsing page... 190
parsing page... 200
parsing page... 210
parsing page... 220
parsing page... 230


KeyError: 'results'